In [1]:
#Some usefull Pyspark Tips

In [1]:
import findspark
findspark.init('/Users/rodri/Desktop/Spark/spark-3.0.0-preview2-bin-hadoop2.7')
import pyspark
from pyspark import SparkConf , SparkContext
from pyspark.sql import SparkSession,functions
from pyspark.sql import *
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
from pyspark.sql.functions import lit
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.sql.functions import format_number
from pyspark.sql.functions import first
from pyspark.sql.functions import count
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType

config = SparkConf()
sc = SparkContext.getOrCreate(conf=config)
sqlContext = SQLContext(sc)
spark = SparkSession.builder.getOrCreate()

#### Creating Spark DataFrames - Criando DataFrames Spark

In [3]:
data = [("Preco",35,"VideoGames"),
        ("Peter",31,"Pizzaiolo"),
        ("Felipe",26,"RichStuff")]
df = spark.createDataFrame(data, ["Nome","Idade","Hobby"])

In [4]:
#Show dataframe
#Mostrar dataframe
df.show()

+------+-----+----------+
|  Nome|Idade|     Hobby|
+------+-----+----------+
| Preco|   35|VideoGames|
| Peter|   31| Pizzaiolo|
|Felipe|   26| RichStuff|
+------+-----+----------+



In [5]:
df.printSchema()

root
 |-- Nome: string (nullable = true)
 |-- Idade: long (nullable = true)
 |-- Hobby: string (nullable = true)



In [6]:
df.columns

['Nome', 'Idade', 'Hobby']

In [7]:
df.describe().show()

+-------+------+------------------+----------+
|summary|  Nome|             Idade|     Hobby|
+-------+------+------------------+----------+
|  count|     3|                 3|         3|
|   mean|  null|30.666666666666668|      null|
| stddev|  null| 4.509249752822894|      null|
|    min|Felipe|                26| Pizzaiolo|
|    max| Preco|                35|VideoGames|
+-------+------+------------------+----------+



#### Schema Defining - Definindo o Schema

In [8]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [9]:
#StructField -> Nome do campo, tipo do campo, True (pode conter nulos)
data_schema = [
               StructField('Nome',StringType(),True),
               StructField('Idade',IntegerType(),True),
               StructField('Hobby',StringType(),True)
              ]

In [10]:
final_struc = StructType(fields=data_schema)

In [11]:
data = [("Preco",35,"VideoGames"),
        ("Peter",31,"Pizzaiolo"),
        ("Felipe",26,"RichStuff")]
df = spark.createDataFrame(data, ["Nome","Idade","Hobby"],final_struc)

In [12]:
df.show()

+------+-----+----------+
|  Nome|Idade|     Hobby|
+------+-----+----------+
| Preco|   35|VideoGames|
| Peter|   31| Pizzaiolo|
|Felipe|   26| RichStuff|
+------+-----+----------+



In [13]:
df.printSchema()

root
 |-- Nome: string (nullable = true)
 |-- Idade: long (nullable = true)
 |-- Hobby: string (nullable = true)



#### Spark Dataframe Stuff

In [14]:
df.select('Idade').show()

+-----+
|Idade|
+-----+
|   35|
|   31|
|   26|
+-----+



In [15]:
df.head(3)

[Row(Nome='Preco', Idade=35, Hobby='VideoGames'),
 Row(Nome='Peter', Idade=31, Hobby='Pizzaiolo'),
 Row(Nome='Felipe', Idade=26, Hobby='RichStuff')]

In [16]:
df.head(3)[0]

Row(Nome='Preco', Idade=35, Hobby='VideoGames')

In [17]:
df.select(['Idade','Nome']).show()

+-----+------+
|Idade|  Nome|
+-----+------+
|   35| Preco|
|   31| Peter|
|   26|Felipe|
+-----+------+



In [18]:
#Add columns
#Adicionar colunas
df.withColumn('Idade_x2',df['Idade']*2).show()

+------+-----+----------+--------+
|  Nome|Idade|     Hobby|Idade_x2|
+------+-----+----------+--------+
| Preco|   35|VideoGames|      70|
| Peter|   31| Pizzaiolo|      62|
|Felipe|   26| RichStuff|      52|
+------+-----+----------+--------+



In [19]:
#Add a hardcoded column value
#Criando uma coluna chumbada
from pyspark.sql.functions import lit

df.withColumn("hardcoded_colum", lit("Value_Chumbado")).show()

+------+-----+----------+---------------+
|  Nome|Idade|     Hobby|hardcoded_colum|
+------+-----+----------+---------------+
| Preco|   35|VideoGames| Value_Chumbado|
| Peter|   31| Pizzaiolo| Value_Chumbado|
|Felipe|   26| RichStuff| Value_Chumbado|
+------+-----+----------+---------------+



In [20]:
#The code above doesnot inplaced the original df
#Veja que o código a cima não deu "inplace" no dataframe
df.show()

+------+-----+----------+
|  Nome|Idade|     Hobby|
+------+-----+----------+
| Preco|   35|VideoGames|
| Peter|   31| Pizzaiolo|
|Felipe|   26| RichStuff|
+------+-----+----------+



In [21]:
#inplacing created column
df = df.withColumn('Idade_x2',df['Idade']*2)

In [22]:
df.show()

+------+-----+----------+--------+
|  Nome|Idade|     Hobby|Idade_x2|
+------+-----+----------+--------+
| Preco|   35|VideoGames|      70|
| Peter|   31| Pizzaiolo|      62|
|Felipe|   26| RichStuff|      52|
+------+-----+----------+--------+



In [23]:
#renaming
df = df.withColumnRenamed('Idade','Idade_renamed')

In [24]:
df.show()

+------+-------------+----------+--------+
|  Nome|Idade_renamed|     Hobby|Idade_x2|
+------+-------------+----------+--------+
| Preco|           35|VideoGames|      70|
| Peter|           31| Pizzaiolo|      62|
|Felipe|           26| RichStuff|      52|
+------+-------------+----------+--------+



In [25]:
#drop columns
df = df.drop("Idade_x2")

In [26]:
df.show()

+------+-------------+----------+
|  Nome|Idade_renamed|     Hobby|
+------+-------------+----------+
| Preco|           35|VideoGames|
| Peter|           31| Pizzaiolo|
|Felipe|           26| RichStuff|
+------+-------------+----------+



#### SQL inside Pyspark - Very usefull

In [27]:
#SQL
df.createOrReplaceTempView('DevDataX')

In [28]:
dataframe = spark.sql("""SELECT * FROM DevDataX""")

In [29]:
dataframe.show()

+------+-------------+----------+
|  Nome|Idade_renamed|     Hobby|
+------+-------------+----------+
| Preco|           35|VideoGames|
| Peter|           31| Pizzaiolo|
|Felipe|           26| RichStuff|
+------+-------------+----------+



In [30]:
new_dataframe = spark.sql("""SELECT * FROM DevDataX WHERE Idade_renamed=26""")

In [31]:
new_dataframe.show()

+------+-------------+---------+
|  Nome|Idade_renamed|    Hobby|
+------+-------------+---------+
|Felipe|           26|RichStuff|
+------+-------------+---------+



In [32]:
df = spark.read.csv("appl_stock.csv",inferSchema=True,header=True)

In [33]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [34]:
df.show(5)

+----------+----------+----------+------------------+------------------+---------+------------------+
|      Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+----------+----------+----------+------------------+------------------+---------+------------------+
only showing top 5 rows



#### Summarizing DataFrames - Resumindo DataFrames 

In [35]:
df.createOrReplaceTempView("df")

In [36]:
#Em sql
#A = spark.sql("""SELECT * FROM df WHERE close < 500""").show()
#Em spark dataframe
df.filter("Close < 500").show(5)

+----------+----------+----------+------------------+------------------+---------+------------------+
|      Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+----------+----------+----------+------------------+------------------+---------+------------------+
only showing top 5 rows



In [37]:
df.filter("Close < 500").select(['open','close']).show(5)

+----------+------------------+
|      open|             close|
+----------+------------------+
|213.429998|        214.009998|
|214.599998|        214.379993|
|214.379993|        210.969995|
|    211.75|            210.58|
|210.299994|211.98000499999998|
+----------+------------------+
only showing top 5 rows



In [38]:
df.filter(df['Close'] < 500).select("Volume").show(5)

+---------+
|   Volume|
+---------+
|123432400|
|150476200|
|138040000|
|119282800|
|111902700|
+---------+
only showing top 5 rows



In [39]:
#Condições Multiplas (& : and, | : or)
df.filter((df['Close'] < 200) & (df['Open'] > 200)).show(5)

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [40]:
#Condições Multiplas (& : and, | : or)
df.filter((df['Close'] < 200) & ~(df['Open'] > 200)).show(5)

+----------+------------------+----------+------------------+----------+---------+------------------+
|      Date|              Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+------------------+----------+---------+------------------+
|2010-02-01|192.36999699999998|     196.0|191.29999899999999|194.729998|187469100|         25.229131|
|2010-02-02|        195.909998|196.319994|193.37999299999998|195.859997|174585600|25.375532999999997|
|2010-02-03|        195.169994|200.200003|        194.420004|199.229994|153832000|25.812148999999998|
|2010-02-04|        196.730003|198.370001|        191.570005|192.050003|189413000|         24.881912|
|2010-02-05|192.63000300000002|     196.0|        190.850002|195.460001|212576700|25.323710000000002|
+----------+------------------+----------+------------------+----------+---------+------------------+
only showing top 5 rows



In [41]:
result = df.filter(df['Low'] == 197.16).collect()

In [42]:
row = result[0]

In [43]:
row.asDict()['Volume']

220441900

In [44]:
df.collect()[0].asDict()['Date']

'2010-01-04'

#### Pyspark Agregations -  Agregações

In [45]:
df = spark.read.csv("sales_info.csv",inferSchema=True,header=True)

In [46]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [47]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [48]:
#mean(), sum(), max(), min(), count()
df.groupby("Company").mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   GOOG|            220.0|
|   MSFT|322.3333333333333|
|     FB|            610.0|
|   APPL|            370.0|
+-------+-----------------+



In [49]:
#Coluna que vc quer agregar : tipo de agregacao
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [50]:
group_data = df.groupby("Company")

In [51]:
group_data.agg({'Sales':'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   GOOG|     340.0|
|   MSFT|     600.0|
|     FB|     870.0|
|   APPL|     750.0|
+-------+----------+



In [52]:
df.groupby("Company").agg({'Sales':'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   GOOG|     340.0|
|   MSFT|     600.0|
|     FB|     870.0|
|   APPL|     750.0|
+-------+----------+



In [53]:
from pyspark.sql.functions import countDistinct,avg,stddev

In [54]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [55]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [56]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [57]:
df.select(avg('Sales').alias('Average_Sales')).show()

+-----------------+
|    Average_Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [58]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [59]:
from pyspark.sql.functions import format_number

In [60]:
sales_std = df.select(stddev('Sales').alias('std'))

In [61]:
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [62]:
#Formatação de números para 2 casas decimais
sales_std.select(format_number('std',2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



In [63]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [64]:
#Order by Ascending
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [65]:
#Order by Descending
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [66]:
from pyspark.sql.functions import first

In [67]:
from pyspark.sql.functions import count

In [68]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



### Summarizing values in one single list value 

In [69]:
data = [("Preco","Barraco"),
        ("Peter","Sítio"),
        ("Peter","CasaPraia"),
        ("Peter","Apartment"),
        ("Felipe","Triplex"),
        ("Felipe","Fazenda")]
df = spark.createDataFrame(data, ["Nome","Imóveis"])

In [70]:
df.groupby('Nome').agg(functions.collect_set("Imóveis").alias("Imóveis_List")).show()

+------+--------------------+
|  Nome|        Imóveis_List|
+------+--------------------+
|Felipe|  [Fazenda, Triplex]|
| Peter|[Sítio, CasaPraia...|
| Preco|           [Barraco]|
+------+--------------------+



### Pivot table - Columns to lines - Colunas para linhas

In [71]:
df = spark.read.csv("sales_info.csv",inferSchema=True,header=True)

In [72]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [73]:
df.groupBy('Company').pivot('Person').agg(first('Person')).show()

+-------+------+----+----+-------+-----+----+-----+----+----+-----+----+-------+
|Company| Chris| Amy|Carl|Charlie|Frank|John|Linda|Mike| Sam|Sarah|Tina|Vanessa|
+-------+------+----+----+-------+-----+----+-----+----+----+-----+----+-------+
|   APPL| Chris|null|null|   null| null|John|Linda|Mike|null| null|null|   null|
|     FB|  null|null|Carl|   null| null|null| null|null|null|Sarah|null|   null|
|   GOOG|  null|null|null|Charlie|Frank|null| null|null| Sam| null|null|   null|
|   MSFT|  null| Amy|null|   null| null|null| null|null|null| null|Tina|Vanessa|
+-------+------+----+----+-------+-----+----+-----+----+----+-----+----+-------+



### Melt table - lines to columns - linhas para colunas

In [2]:
data = [("Preco","Barraco","JAN"),
        ("Peter","Sítio","FEV"),
        ("Peter","CasaPraia","JAN"),
        ("Peter","Apartment","JAN"),
        ("Felipe","Triplex","FEV"),
        ("Felipe","Fazenda","FEV")]
df = spark.createDataFrame(data, ["Nome","Imóveis","Purchase_Month"])

In [3]:
df.show()

+------+---------+--------------+
|  Nome|  Imóveis|Purchase_Month|
+------+---------+--------------+
| Preco|  Barraco|           JAN|
| Peter|    Sítio|           FEV|
| Peter|CasaPraia|           JAN|
| Peter|Apartment|           JAN|
|Felipe|  Triplex|           FEV|
|Felipe|  Fazenda|           FEV|
+------+---------+--------------+



In [7]:
from pyspark.sql.functions import split, explode,array,struct,col
def to_explode(df, by):      
    # Filter dtypes and split into column names and type description
    #Filtrando dtypes e separando em nome de coluna e tipo de coluna
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))     
    # Spark SQL supports only homogeneous columns     
    assert len(set(dtypes)) == 1, "As colunas devem ter o mesmo tipo"
    # Create and explode an array of (column_name, column_value) structs 
    resultado = explode(array([struct(lit(c).alias("CATEGORIA"),
                                      col(c).alias("VALOR")) for c in cols])).alias("resultado")      
    return df.select(by + [resultado]).select(by + 
            ["resultado.CATEGORIA", "resultado.VALOR"])

In [8]:
df2=to_explode(df=df, by=['Nome','Purchase_Month'])

In [9]:
df2.show()

+------+--------------+---------+---------+
|  Nome|Purchase_Month|CATEGORIA|    VALOR|
+------+--------------+---------+---------+
| Preco|           JAN|  Imóveis|  Barraco|
| Peter|           FEV|  Imóveis|    Sítio|
| Peter|           JAN|  Imóveis|CasaPraia|
| Peter|           JAN|  Imóveis|Apartment|
|Felipe|           FEV|  Imóveis|  Triplex|
|Felipe|           FEV|  Imóveis|  Fazenda|
+------+--------------+---------+---------+



### UDF Functions Register - Registrando funções do usuário

In [79]:
data = [("Preco",35,"VideoGames"),
        ("Peter",31,"Pizzaiolo"),
        ("Felipe",26,"RichStuff")]
df = spark.createDataFrame(data, ["Nome","Idade","Hobby"])

In [80]:
df.show()

+------+-----+----------+
|  Nome|Idade|     Hobby|
+------+-----+----------+
| Preco|   35|VideoGames|
| Peter|   31| Pizzaiolo|
|Felipe|   26| RichStuff|
+------+-----+----------+



In [81]:
df.printSchema()

root
 |-- Nome: string (nullable = true)
 |-- Idade: long (nullable = true)
 |-- Hobby: string (nullable = true)



In [82]:
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType

import math

def raiz_quadrada(x):
    return math.sqrt(x)

spark.udf.register("raiz_quadrada", raiz_quadrada)

<function __main__.raiz_quadrada(x)>

In [83]:
df.createOrReplaceTempView('DevDataX')

In [84]:
df2 = spark.sql("""
                    SELECT 
                    Nome,
                    Idade,
                    Hobby,
                    raiz_quadrada(Idade) as Raiz_Idade
                    
                    FROM DevDataX
""").show()

+------+-----+----------+------------------+
|  Nome|Idade|     Hobby|        Raiz_Idade|
+------+-----+----------+------------------+
| Preco|   35|VideoGames| 5.916079783099616|
| Peter|   31| Pizzaiolo|5.5677643628300215|
|Felipe|   26| RichStuff|5.0990195135927845|
+------+-----+----------+------------------+



In [ ]:
import findspark
findspark.init('/Users/rodri/Desktop/Spark/spark-3.0.0-preview2-bin-hadoop2.7')
import pyspark
from pyspark import SparkConf , SparkContext
from pyspark.sql import SparkSession,functions
from pyspark.sql import *
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
from pyspark.sql.functions import lit
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.sql.functions import format_number
from pyspark.sql.functions import first
from pyspark.sql.functions import count
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType
​
config = SparkConf()
sc = SparkContext.getOrCreate(conf=config)
sqlContext = SQLContext(sc)
spark = SparkSession.builder.getOrCreate()
Creating Spark DataFrames - Criando DataFrames Spark
data = [("Preco",35,"VideoGames"),
        ("Peter",31,"Pizzaiolo"),
        ("Felipe",26,"RichStuff")]
df = spark.createDataFrame(data, ["Nome","Idade","Hobby"])
#Show dataframe
#Mostrar dataframe
df.show()
+------+-----+----------+
|  Nome|Idade|     Hobby|
+------+-----+----------+
| Preco|   35|VideoGames|
| Peter|   31| Pizzaiolo|
|Felipe|   26| RichStuff|
+------+-----+----------+

df.printSchema()
df.printSchema()
root
 |-- Nome: string (nullable = true)
 |-- Idade: long (nullable = true)
 |-- Hobby: string (nullable = true)

df.columns
df.columns
['Nome', 'Idade', 'Hobby']
df.describe().show()
+-------+------+------------------+----------+
|summary|  Nome|             Idade|     Hobby|
+-------+------+------------------+----------+
|  count|     3|                 3|         3|
|   mean|  null|30.666666666666668|      null|
| stddev|  null| 4.509249752822894|      null|
|    min|Felipe|                26| Pizzaiolo|
|    max| Preco|                35|VideoGames|
+-------+------+------------------+----------+

Schema Defining - Definindo o Schema
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
#StructField -> Nome do campo, tipo do campo, True (pode conter nulos)
data_schema = [
               StructField('Nome',StringType(),True),
               StructField('Idade',IntegerType(),True),
               StructField('Hobby',StringType(),True)
              ]
final_struc = StructType(fields=data_schema)
data = [("Preco",35,"VideoGames"),
        ("Peter",31,"Pizzaiolo"),
        ("Felipe",26,"RichStuff")]
df = spark.createDataFrame(data, ["Nome","Idade","Hobby"],final_struc)
df.show()
+------+-----+----------+
|  Nome|Idade|     Hobby|
+------+-----+----------+
| Preco|   35|VideoGames|
| Peter|   31| Pizzaiolo|
|Felipe|   26| RichStuff|
+------+-----+----------+

df.printSchema()
root
 |-- Nome: string (nullable = true)
 |-- Idade: long (nullable = true)
 |-- Hobby: string (nullable = true)

Spark Dataframe Stuff
df.select('Idade').show()
+-----+
|Idade|
+-----+
|   35|
|   31|
|   26|
+-----+

df.head(3)
[Row(Nome='Preco', Idade=35, Hobby='VideoGames'),
 Row(Nome='Peter', Idade=31, Hobby='Pizzaiolo'),
 Row(Nome='Felipe', Idade=26, Hobby='RichStuff')]
df.head(3)[0]
Row(Nome='Preco', Idade=35, Hobby='VideoGames')
df.select(['Idade','Nome']).show()
+-----+------+
|Idade|  Nome|
+-----+------+
|   35| Preco|
|   31| Peter|
|   26|Felipe|
+-----+------+

#Add columns
#Adicionar colunas
df.withColumn('Idade_x2',df['Idade']*2).show()
+------+-----+----------+--------+
|  Nome|Idade|     Hobby|Idade_x2|
+------+-----+----------+--------+
| Preco|   35|VideoGames|      70|
| Peter|   31| Pizzaiolo|      62|
|Felipe|   26| RichStuff|      52|
+------+-----+----------+--------+

from pyspark.sql.functions import lit
#Add a hardcoded column value
#Criando uma coluna chumbada
from pyspark.sql.functions import lit
​
df.withColumn("hardcoded_colum", lit("Value_Chumbado")).show()
+------+-----+----------+---------------+
|  Nome|Idade|     Hobby|hardcoded_colum|
+------+-----+----------+---------------+
| Preco|   35|VideoGames| Value_Chumbado|
| Peter|   31| Pizzaiolo| Value_Chumbado|
|Felipe|   26| RichStuff| Value_Chumbado|
+------+-----+----------+---------------+

#The code above doesnot inplaced the original df
#Veja que o código a cima não deu "inplace" no dataframe
df.show()
+------+-----+----------+
|  Nome|Idade|     Hobby|
+------+-----+----------+
| Preco|   35|VideoGames|
| Peter|   31| Pizzaiolo|
|Felipe|   26| RichStuff|
+------+-----+----------+

#inplacing created column
df = df.withColumn('Idade_x2',df['Idade']*2)
df.show()
+------+-----+----------+--------+
|  Nome|Idade|     Hobby|Idade_x2|
+------+-----+----------+--------+
| Preco|   35|VideoGames|      70|
| Peter|   31| Pizzaiolo|      62|
|Felipe|   26| RichStuff|      52|
+------+-----+----------+--------+

#renaming
df = df.withColumnRenamed('Idade','Idade_renamed')
df.show()
+------+-------------+----------+--------+
|  Nome|Idade_renamed|     Hobby|Idade_x2|
+------+-------------+----------+--------+
| Preco|           35|VideoGames|      70|
| Peter|           31| Pizzaiolo|      62|
|Felipe|           26| RichStuff|      52|
+------+-------------+----------+--------+

#drop columns
df = df.drop("Idade_x2")
df.show()
+------+-------------+----------+
|  Nome|Idade_renamed|     Hobby|
+------+-------------+----------+
| Preco|           35|VideoGames|
| Peter|           31| Pizzaiolo|
|Felipe|           26| RichStuff|
+------+-------------+----------+

SQL inside Pyspark - Very usefull
#SQL
df.createOrReplaceTempView('DevDataX')
dataframe = spark.sql("""SELECT * FROM DevDataX""")
dataframe.show()
+------+-------------+----------+
|  Nome|Idade_renamed|     Hobby|
+------+-------------+----------+
| Preco|           35|VideoGames|
| Peter|           31| Pizzaiolo|
|Felipe|           26| RichStuff|
+------+-------------+----------+

new_dataframe = spark.sql("""SELECT * FROM DevDataX WHERE Idade_renamed=26""")
new_dataframe.show()
+------+-------------+---------+
|  Nome|Idade_renamed|    Hobby|
+------+-------------+---------+
|Felipe|           26|RichStuff|
+------+-------------+---------+

df = spark.read.csv("appl_stock.csv",inferSchema=True,header=True)
df.printSchema()
root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

df.show(5)
+----------+----------+----------+------------------+------------------+---------+------------------+
|      Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+----------+----------+----------+------------------+------------------+---------+------------------+
only showing top 5 rows

Summarizing DataFrames - Resumindo DataFrames
df.createOrReplaceTempView("df")
#Em sql
#A = spark.sql("""SELECT * FROM df WHERE close < 500""").show()
#Em spark dataframe
df.filter("Close < 500").show(5)
+----------+----------+----------+------------------+------------------+---------+------------------+
|      Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+----------+----------+----------+------------------+------------------+---------+------------------+
only showing top 5 rows

df.filter("Close < 500").select(['open','close']).show(5)
+----------+------------------+
|      open|             close|
+----------+------------------+
|213.429998|        214.009998|
|214.599998|        214.379993|
|214.379993|        210.969995|
|    211.75|            210.58|
|210.299994|211.98000499999998|
+----------+------------------+
only showing top 5 rows

df.filter(df['Close'] < 500).select("Volume").show(5)
+---------+
|   Volume|
+---------+
|123432400|
|150476200|
|138040000|
|119282800|
|111902700|
+---------+
only showing top 5 rows

#Condições Multiplas (& : and, | : or)
df.filter((df['Close'] < 200) & (df['Open'] > 200)).show(5)
+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+

#Condições Multiplas (& : and, | : or)
df.filter((df['Close'] < 200) & ~(df['Open'] > 200)).show(5)
+----------+------------------+----------+------------------+----------+---------+------------------+
|      Date|              Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+------------------+----------+---------+------------------+
|2010-02-01|192.36999699999998|     196.0|191.29999899999999|194.729998|187469100|         25.229131|
|2010-02-02|        195.909998|196.319994|193.37999299999998|195.859997|174585600|25.375532999999997|
|2010-02-03|        195.169994|200.200003|        194.420004|199.229994|153832000|25.812148999999998|
|2010-02-04|        196.730003|198.370001|        191.570005|192.050003|189413000|         24.881912|
|2010-02-05|192.63000300000002|     196.0|        190.850002|195.460001|212576700|25.323710000000002|
+----------+------------------+----------+------------------+----------+---------+------------------+
only showing top 5 rows

result = df.filter(df['Low'] == 197.16).collect()
row = result[0]
row.asDict()['Volume']
220441900
df.collect()[0].asDict()['Date']
'2010-01-04'
Pyspark Agregations - Agregações
df = spark.read.csv("sales_info.csv",inferSchema=True,header=True)
df.show()
+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+

df.printSchema()
root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)

#mean(), sum(), max(), min(), count()
df.groupby("Company").mean().show()
+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   GOOG|            220.0|
|   MSFT|322.3333333333333|
|     FB|            610.0|
|   APPL|            370.0|
+-------+-----------------+

#Coluna que vc quer agregar : tipo de agregacao
df.agg({'Sales':'sum'}).show()
+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+

group_data = df.groupby("Company")
group_data.agg({'Sales':'max'}).show()
+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   GOOG|     340.0|
|   MSFT|     600.0|
|     FB|     870.0|
|   APPL|     750.0|
+-------+----------+

df.groupby("Company").agg({'Sales':'max'}).show()
+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   GOOG|     340.0|
|   MSFT|     600.0|
|     FB|     870.0|
|   APPL|     750.0|
+-------+----------+

from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.sql.functions import countDistinct,avg,stddev
df.select(countDistinct('Sales')).show()
+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+

df.select(avg('Sales')).show()
+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+

df.select(avg('Sales')).show()
+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+

df.select(avg('Sales').alias('Average_Sales')).show()
+-----------------+
|    Average_Sales|
+-----------------+
|360.5833333333333|
+-----------------+

df.select(stddev('Sales')).show()
+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+

from pyspark.sql.functions import format_number
from pyspark.sql.functions import format_number
sales_std = df.select(stddev('Sales').alias('std'))
sales_std.show()
+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+

#Formatação de números para 2 casas decimais
sales_std.select(format_number('std',2).alias('std')).show()
+------+
|   std|
+------+
|250.09|
+------+

df.show()
+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+

#Order by Ascending
df.orderBy('Sales').show()
+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+

#Order by Descending
df.orderBy(df['Sales'].desc()).show()
+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

from pyspark.sql.functions import first
from pyspark.sql.functions import count
df.show()
+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+

Summarizing values in one single list value
data = [("Preco","Barraco"),
        ("Peter","Sítio"),
        ("Peter","CasaPraia"),
        ("Peter","Apartment"),
        ("Felipe","Triplex"),
        ("Felipe","Fazenda")]
df = spark.createDataFrame(data, ["Nome","Imóveis"])
df.groupby('Nome').agg(functions.collect_set("Imóveis").alias("Imóveis_List")).show()
+------+--------------------+
|  Nome|        Imóveis_List|
+------+--------------------+
|Felipe|  [Fazenda, Triplex]|
| Peter|[Sítio, CasaPraia...|
| Preco|           [Barraco]|
+------+--------------------+

Pivot table - Columns to lines - Colunas para linhas
df = spark.read.csv("sales_info.csv",inferSchema=True,header=True)
df.show()
+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+

df.groupBy('Company').pivot('Person').agg(first('Person')).show()
+-------+------+----+----+-------+-----+----+-----+----+----+-----+----+-------+
|Company| Chris| Amy|Carl|Charlie|Frank|John|Linda|Mike| Sam|Sarah|Tina|Vanessa|
+-------+------+----+----+-------+-----+----+-----+----+----+-----+----+-------+
|   APPL| Chris|null|null|   null| null|John|Linda|Mike|null| null|null|   null|
|     FB|  null|null|Carl|   null| null|null| null|null|null|Sarah|null|   null|
|   GOOG|  null|null|null|Charlie|Frank|null| null|null| Sam| null|null|   null|
|   MSFT|  null| Amy|null|   null| null|null| null|null|null| null|Tina|Vanessa|
+-------+------+----+----+-------+-----+----+-----+----+----+-----+----+-------+

Melt table - lines to columns - linhas para colunas
data = [("Preco","Barraco","JAN"),
        ("Peter","Sítio","FEV"),
        ("Peter","CasaPraia","JAN"),
        ("Peter","Apartment","JAN"),
        ("Felipe","Triplex","FEV"),
        ("Felipe","Fazenda","FEV")]
df = spark.createDataFrame(data, ["Nome","Imóveis","Purchase_Month"])
df.show()
+------+---------+--------------+
|  Nome|  Imóveis|Purchase_Month|
+------+---------+--------------+
| Preco|  Barraco|           JAN|
| Peter|    Sítio|           FEV|
| Peter|CasaPraia|           JAN|
| Peter|Apartment|           JAN|
|Felipe|  Triplex|           FEV|
|Felipe|  Fazenda|           FEV|
+------+---------+--------------+

from pyspark.sql.functions import split, explode,array,struct,col
def to_explode(df, by):      
    # Filter dtypes and split into column names and type description
    #Filtrando dtypes e separando em nome de coluna e tipo de coluna
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))     
    # Spark SQL supports only homogeneous columns     
    assert len(set(dtypes)) == 1, "As colunas devem ter o mesmo tipo"
    # Create and explode an array of (column_name, column_value) structs 
    resultado = explode(array([struct(lit(c).alias("CATEGORIA"), col(c).alias("VALOR")) for c in cols])).alias("resultado")      
    return df.select(by + [resultado]).select(by + ["resultado.CATEGORIA", "resultado.VALOR"])
df2=to_explode(df=df, by=['Nome','Purchase_Month'])
df2.show()
+------+--------------+---------+---------+
|  Nome|Purchase_Month|CATEGORIA|    VALOR|
+------+--------------+---------+---------+
| Preco|           JAN|  Imóveis|  Barraco|
| Peter|           FEV|  Imóveis|    Sítio|
| Peter|           JAN|  Imóveis|CasaPraia|
| Peter|           JAN|  Imóveis|Apartment|
|Felipe|           FEV|  Imóveis|  Triplex|
|Felipe|           FEV|  Imóveis|  Fazenda|
+------+--------------+---------+---------+

UDF Functions Register - Registrando funções do usuário
data = [("Preco",35,"VideoGames"),
        ("Peter",31,"Pizzaiolo"),
        ("Felipe",26,"RichStuff")]
df = spark.createDataFrame(data, ["Nome","Idade","Hobby"])
df.show()
+------+-----+----------+
|  Nome|Idade|     Hobby|
+------+-----+----------+
| Preco|   35|VideoGames|
| Peter|   31| Pizzaiolo|
|Felipe|   26| RichStuff|
+------+-----+----------+

df.printSchema()
root
 |-- Nome: string (nullable = true)
 |-- Idade: long (nullable = true)
 |-- Hobby: string (nullable = true)

from pyspark.sql.functions import udf
from pyspark.sql.types import LongType
​
import math
​
def raiz_quadrada(x):
    return math.sqrt(x)
​
spark.udf.register("raiz_quadrada", raiz_quadrada)
<function __main__.raiz_quadrada(x)>
df.createOrReplaceTempView('DevDataX')
df2 = spark.sql("""

df2 = spark.sql("""
                    SELECT 
                    Nome,
                    Idade,
                    Hobby,
                    raiz_quadrada(Idade) as Raiz_Idade
                    
                    FROM DevDataX
""").show()
+------+-----+----------+------------------+
|  Nome|Idade|     Hobby|        Raiz_Idade|
+------+-----+----------+------------------+
| Preco|   35|VideoGames| 5.916079783099616|
| Peter|   31| Pizzaiolo|5.5677643628300215|
|Felipe|   26| RichStuff|5.0990195135927845|
+------+-----+----------+------------------+